In [47]:
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import keyring
import time

## 0. Setup Spotipy credentials and query wrapper

In [48]:
client_credentials_manager = SpotifyClientCredentials(client_id=keyring.get_password('spotify', 'cid'),
                                                      client_secret=keyring.get_password('spotify', 'secret') )
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


Set keyword

In [49]:
KEYWORD = 'R&B'

## 1. Search for the top N playlists for keyword

##### View the structure of a search query

In [50]:
results = sp.search(q=KEYWORD, type='playlist', market='PH')

In [51]:
[r['name'] for r in results['playlists']['items']]

["I Love My '00s R&B",
 'Are & Be',
 'R&B Hits 2020',
 'R&B love songs 1990-2020',
 'R&B Favourites',
 'R&B 2000s Greatest hits',
 "R&B 2000's mix",
 "R&B Music You'll Love",
 'r&b mix',
 '2000 R&B Hits']

In [52]:
results['playlists'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [53]:
results['playlists']['items'][0].keys()

dict_keys(['collaborative', 'description', 'external_urls', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

***

In [54]:
playlist_ids = []
playlist_names = []
playlist_numtracks = []

N = 100
#get playlist in batches of 50
for n in np.arange(N//50):
    offset= 50*n
    print("Getting batch %d of search results for keyword: %s ..." % (n,KEYWORD), end='' )
    results = sp.search(q=KEYWORD, type='playlist' , market='PH', offset = offset, limit=50)
    playlist_ids.extend([p['href'].split('/')[5] for p in results['playlists']['items']])
    playlist_names.extend([p['name'] for p in results['playlists']['items']])
    playlist_numtracks.extend([p['tracks']['total'] for p in results['playlists']['items']])
    print("  DONE!")

Getting batch 0 of search results for keyword: R&B ...  DONE!
Getting batch 1 of search results for keyword: R&B ...  DONE!


In [55]:
playlist_names

["I Love My '00s R&B",
 'Are & Be',
 'R&B Hits 2020',
 'R&B love songs 1990-2020',
 'R&B Favourites',
 'R&B 2000s Greatest hits',
 "R&B 2000's mix",
 "R&B Music You'll Love",
 'r&b mix',
 '2000 R&B Hits',
 'R&B Gems',
 '⭐"R&B"',
 "I Love My '90s R&B",
 'R&B',
 'TrenChill K-R&B',
 'R&B 2020',
 'Chilled R&B',
 'Pinas Vibes',
 'R & B',
 'R&B Workout',
 'r&b',
 'R&B Playlist 2020 - Today’s TOP R&B Hits (New RnB Songs Playlist)',
 'R&B Party Classics',
 'R&B 2020 - Best R&B Songs Playlist (RNB Music 2020)',
 "R&B & HipHop 2000's",
 ' R&B Throwback (90’s-00’s)',
 'R&B em Casa',
 'R&B Hip Hop Rap Soul',
 'Alternative R&B',
 'R&B/THROWBACK JAMS',
 'Who We Be',
 'B.A.E.',
 'Run This Town 150–165 BPM',
 '’80s Jam Session',
 'The Newness',
 'R&B Right Now',
 'R&Bae',
 'Bollywood R&B',
 'This Is How We Do',
 'R&B Connect',
 'Vibes',
 'Chicha Royale',
 'R&B Classics',
 'Suprême R&B',
 "I Love My '10s R&B",
 'Habibi Gang Gang',
 'Fresh Finds: The Wave',
 "'90s Baby Makers",
 'The Flavour',
 'Hip Hop

## 2. Get Playlist Data

##### View the structure of a playlist query

In [56]:
playlist = sp.playlist('37i9dQZF1DX4olOMiqFeqU')

In [57]:
playlist

{'collaborative': False,
 'description': '100% Pinoy hits from your favorite artists!',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX4olOMiqFeqU'},
 'followers': {'href': None, 'total': 1990882},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX4olOMiqFeqU?additional_types=track',
 'id': '37i9dQZF1DX4olOMiqFeqU',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab67706f00000003686604cfc2467f94dfcd8e3a',
   'width': None}],
 'name': 'OPM Favorites',
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': False,
 'snapshot_id': 'MTYwMjM2MTA1NSwwMDAwMDAwMGYyZGQ1NmM3ZGJiZGQ5MmQ5MTI5ZTg4MWYyMWIzNWY1',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX4olOMiqFeqU/tracks?offset=0&limit=100&additional_types=tr

In [58]:
playlist.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

***

In [59]:
playlist_lookup = []
for n,p_id in enumerate(playlist_ids):
    print("Getting playlist data for playlist %s :..." % (playlist_names[n]), end='' )
    playlist = sp.playlist(p_id)
    try:
        relevant_playlist_data = { key: playlist[key] for key in ['followers','owner']}
        relevant_playlist_data['playlist_id'] = p_id
        relevant_playlist_data['playlist_name'] = playlist_names[n]
        relevant_playlist_data['playlist_total_tracks'] = playlist_numtracks[n]
        relevant_playlist_data['owner_id'] = playlist['owner']['id']
        relevant_playlist_data['owner_name'] = playlist['owner']['display_name']
        relevant_playlist_data['total_followers'] = playlist['followers']['total']
        relevant_playlist_data.pop('owner', None)
        relevant_playlist_data.pop('followers', None)
        playlist_lookup.append(relevant_playlist_data)
        print("   DONE")
    except:
        print("   Aborted")
        continue
    

Getting playlist data for playlist I Love My '00s R&B :...   DONE
Getting playlist data for playlist Are & Be :...   DONE
Getting playlist data for playlist R&B Hits 2020 :...   DONE
Getting playlist data for playlist R&B love songs 1990-2020 :...   DONE
Getting playlist data for playlist R&B Favourites :...   DONE
Getting playlist data for playlist R&B 2000s Greatest hits :...   DONE
Getting playlist data for playlist R&B 2000's mix :...   DONE
Getting playlist data for playlist R&B Music You'll Love :...   DONE
Getting playlist data for playlist r&b mix :...   DONE
Getting playlist data for playlist 2000 R&B Hits :...   DONE
Getting playlist data for playlist R&B Gems :...   DONE
Getting playlist data for playlist ⭐"R&B" :...   DONE
Getting playlist data for playlist I Love My '90s R&B :...   DONE
Getting playlist data for playlist R&B :...   DONE
Getting playlist data for playlist TrenChill K-R&B :...   DONE
Getting playlist data for playlist R&B 2020 :...   DONE
Getting playlist da

In [60]:
playlist_df = pd.DataFrame(playlist_lookup)
playlist_df =playlist_df.sort_values('total_followers',ascending=False)
playlist_df 

,playlist_id,playlist_name,playlist_total_tracks,owner_id,owner_name,total_followers
1,37i9dQZF1DX4SBhb3fqCJd,Are & Be,35,spotify,Spotify,5414770
0,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B,40,spotify,Spotify,3186018
12,37i9dQZF1DX6VDO8a6cQME,I Love My '90s R&B,45,spotify,Spotify,2386964
16,37i9dQZF1DX2UgsUIg75Vg,Chilled R&B,50,spotify,Spotify,2030733
31,37i9dQZF1DWX3387IZmjNa,B.A.E.,50,spotify,Spotify,1795394
...,...,...,...,...,...,...
7,6vlSrYqcEFui4UvAmBLsPD,R&B Music You'll Love,84,l13xuooc8wbyl22qr15457pp4,Music You'll Love,696
15,6YxrDeBWISYdJnTu4zyjL9,R&B 2020,104,31fudjrnhfiuqv5jexx6lvkqibnm,Miguel Alba,273
8,36Q3bcqGA0AeKQO9mshntX,r&b mix,101,kgky0kxdxbjrorxcjnm0bpmfu,cnuzzolo123,239
26,4IypUdqK2We1KU1wFZ1RTT,R&B em Casa,20,6ol1iut542g2pyd2v1wng43on,richards74,97


In [61]:
#playlist name must contain the keyword
playlist_df = playlist_df[playlist_df['playlist_name'].str.lower().str.contains(KEYWORD.lower())]

In [62]:
playlist_df.to_csv("data/"+KEYWORD+"_playlist_data.csv",encoding='utf=8',index=False)

## 3. Get Tracks from a Playlist

##### View the structure of a playlist_tracks query

In [63]:
track = sp.playlist_tracks('37i9dQZF1DX4olOMiqFeqU')

In [64]:
track

{'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX4olOMiqFeqU/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2020-10-05T08:50:05Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7374lH6kwx9uQATYQ9H3Cp'},
       'href': 'https://api.spotify.com/v1/artists/7374lH6kwx9uQATYQ9H3Cp',
       'id': '7374lH6kwx9uQATYQ9H3Cp',
       'name': 'Eraserheads',
       'type': 'artist',
       'uri': 'spotify:artist:7374lH6kwx9uQATYQ9H3Cp'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',

In [65]:
track.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [66]:
track['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

***

In [67]:
#get only top 20 followed playlists
playlist_df = playlist_df.head(20)

In [68]:
def get_relevant_track_data(tracks_data, playlist_id, playlist_name):
    try:
        relevant_track_data = { key: tracks_data['track'][key] for key in ['id','artists','name','popularity','duration_ms'] }
        relevant_track_data['artist_id']=[artist['id'] for artist in relevant_track_data['artists'] ]
        relevant_track_data['artist_name']=[artist['name']for artist in relevant_track_data['artists'] ]
        relevant_track_data['num_artists']=len([artist['id'] for artist in relevant_track_data['artists']]) 
        relevant_track_data['playlist_id']=playlist_id
        relevant_track_data['playlist_name']=playlist_name
        
        relevant_track_data.pop('artists', None)
        return relevant_track_data
    except:
        return 

In [69]:
#playlist_tracks
all_track_data = []

for _,p_id,p_name, p_numtracks in playlist_df[['playlist_id','playlist_name','playlist_total_tracks']].to_records():
    print("Fetching data for playlist = %s, with total tracks: %d" % (p_name,p_numtracks))
    n_fetches = p_numtracks // 100
    
    playlist_track_data = []
    #get tracks in batches of 100
    for n in np.arange(n_fetches+1):
        track_data = sp.playlist_tracks(p_id, offset=n*100)
        playlist_track_data.extend([get_relevant_track_data(item, p_id,p_name) for item in track_data['items']])
        
    all_track_data.extend(playlist_track_data)

Fetching data for playlist = I Love My '00s R&B, with total tracks: 40
Fetching data for playlist = I Love My '90s R&B, with total tracks: 45
Fetching data for playlist = Chilled R&B, with total tracks: 50
Fetching data for playlist = Alternative R&B, with total tracks: 100
Fetching data for playlist = TrenChill K-R&B, with total tracks: 50
Fetching data for playlist = Women of R&B, with total tracks: 70
Fetching data for playlist = R&B Right Now, with total tracks: 62
Fetching data for playlist = R&B Party Classics, with total tracks: 134
Fetching data for playlist = Suprême R&B, with total tracks: 50
Fetching data for playlist = R&B Connect, with total tracks: 60
Fetching data for playlist = R&B/THROWBACK JAMS, with total tracks: 276
Fetching data for playlist = This Is R&B!🎤, with total tracks: 78
Fetching data for playlist = R&B Brasil, with total tracks: 80
Fetching data for playlist = Sexy R&B, with total tracks: 55
Fetching data for playlist = R&B Favourites, with total tracks: 

In [70]:
for n,a in enumerate(all_track_data):
    try:
        len(a)
    except:
        print(n)

In [71]:
tracks_df = pd.DataFrame([data for data in all_track_data if data is not None])
tracks_df = tracks_df.rename(columns={'id':'track_id'})
tracks_df['artist_id'] = tracks_df.apply(lambda x: x['artist_id'][0] if x['num_artists']==1 else x['artist_id'], axis=1)
tracks_df['artist_name'] = tracks_df.apply(lambda x: x['artist_name'][0] if x['num_artists']==1 else x['artist_name'], axis=1)
tracks_df.head()

,track_id,name,popularity,duration_ms,artist_id,artist_name,num_artists,playlist_id,playlist_name
0,6nTcn8CybBcx0t7IaXEJ6O,Thank God I Found You (feat. Joe & Nas) - Make...,55,251640,"[4iHNK0tOyZPYnBU7nGAgpQ, 3zTOe1BtyTkwNvYZOxXkt...","[Mariah Carey, Joe, Nas, Clue, Supa Engineer ""...",5,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
1,7FWuzwapuKHTEyDJIcQOw3,Stutter (feat. Mystikal) - Double Take Remix,55,213026,"[3zTOe1BtyTkwNvYZOxXktX, 3LIJJJkO7R5RasRwt7xIn...","[Joe, Mystikal, Allen Gordon]",3,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
2,3QE7ObGjkt1kYnhLvZPRUy,Go On Girl,65,261426,21E3waRsmPlU7jZsS13rcj,Ne-Yo,1,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
3,6De63WRULDaa1fxOyLilnv,Stingy,54,259493,7r8RF1tN2A4CiGEplkp1oP,Ginuwine,1,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
4,08fhbfRSg7eEH0IZT7pB6a,That Girl,53,215960,1goOx6gnQdUllLfSMsL4Rt,Marques Houston,1,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B


In [72]:
len(tracks_df)

1685

In [73]:
len(tracks_df['track_id'].unique())

1493

In [74]:
tracks_df.to_csv("data/"+KEYWORD+"_playlist_tracks.csv",encoding='utf=8',index=False)

## 4. Get Tracks from a Playlist

In [75]:
tracks_df = pd.read_csv("data/"+KEYWORD+"_playlist_tracks.csv")
tracks_df.head()

,track_id,name,popularity,duration_ms,artist_id,artist_name,num_artists,playlist_id,playlist_name
0,6nTcn8CybBcx0t7IaXEJ6O,Thank God I Found You (feat. Joe & Nas) - Make...,55,251640,"['4iHNK0tOyZPYnBU7nGAgpQ', '3zTOe1BtyTkwNvYZOx...","['Mariah Carey', 'Joe', 'Nas', 'Clue', 'Supa E...",5,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
1,7FWuzwapuKHTEyDJIcQOw3,Stutter (feat. Mystikal) - Double Take Remix,55,213026,"['3zTOe1BtyTkwNvYZOxXktX', '3LIJJJkO7R5RasRwt7...","['Joe', 'Mystikal', 'Allen Gordon']",3,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
2,3QE7ObGjkt1kYnhLvZPRUy,Go On Girl,65,261426,21E3waRsmPlU7jZsS13rcj,Ne-Yo,1,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
3,6De63WRULDaa1fxOyLilnv,Stingy,54,259493,7r8RF1tN2A4CiGEplkp1oP,Ginuwine,1,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B
4,08fhbfRSg7eEH0IZT7pB6a,That Girl,53,215960,1goOx6gnQdUllLfSMsL4Rt,Marques Houston,1,37i9dQZF1DWYmmr74INQlb,I Love My '00s R&B


In [76]:
tracks_df.shape

(1685, 9)

In [77]:
#remove track duplicates
tracks_df = tracks_df.drop_duplicates(subset='track_id')
tracks_df.shape

(1493, 9)

In [78]:
def get_track_data(t_id, playlist_id,playlist_name):                    
    track_data = sp.track(t_id)
    track_features = sp.audio_features(t_id)
    
    #get only main(first) artist
    td_list = [t_id,\
               track_data['name'],\
               track_data['artists'][0]['id'],\
               track_data['artists'][0]['name'],\
               track_data['album']['uri'].split(":")[2],\
               track_data['duration_ms'],\
               track_data['album']['release_date'],\
               track_data['popularity']]
    data = pd.DataFrame([td_list], columns = ['track_id','track_name','artist_id','artist_name','album_id','duration','release_date','popularity'])

    relevant_cols = ['danceability', 'energy', 'key', 'loudness', 'mode',\
                     'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']  
    
    tf_data = pd.DataFrame(track_features)
    tf_data = tf_data[relevant_cols]
    #tag with source playlist
    tf_data['playlist_id'] = playlist_id
    tf_data['playlist_name'] = playlist_name
    
    data = pd.concat([data, tf_data], axis=1)
    return data


In [79]:
downloaded_track_data = []

In [80]:
track_list = tracks_df['track_id'].values
playlist_name_list = tracks_df['track_id'].values
playlist_id_list = tracks_df['track_id'].values
df_list=[]

for i,track_id in enumerate(track_list):
    try:
        if track_id not in downloaded_track_data:
            print('[%d/%d] Fetching track data for %s... ' % 
                  (i+1,len(track_list),tracks_df[tracks_df['track_id']==track_id]['name'].values[0]), end = " ") 
            track_data = get_track_data(track_id, playlist_id_list[i],playlist_name_list[i]) 
            df_list.append(track_data)
            downloaded_track_data.append(track_id)
            print('done!')
    except:
        continue
    else:
        continue
    
    #sleep for 60 secs per 100 requests to avoid being blocked
    if (i % 100 == 0)&(i > 0):
        time.sleep(20)    

[1/1493] Fetching track data for Thank God I Found You (feat. Joe & Nas) - Make It Last Remix Edit...  done!
[2/1493] Fetching track data for Stutter (feat. Mystikal) - Double Take Remix...  done!
[3/1493] Fetching track data for Go On Girl...  done!
[4/1493] Fetching track data for Stingy...  done!
[5/1493] Fetching track data for That Girl...  done!
[6/1493] Fetching track data for Talkin' To Me (Edit)...  done!
[7/1493] Fetching track data for I Should Have Cheated...  done!
[8/1493] Fetching track data for Shake It Off...  done!
[9/1493] Fetching track data for He Is...  done!
[10/1493] Fetching track data for Do You...  done!
[11/1493] Fetching track data for Into You (feat. Fabolous)...  done!
[12/1493] Fetching track data for He Wasn't Man Enough...  done!
[13/1493] Fetching track data for Touch...  done!
[14/1493] Fetching track data for Love...  done!
[15/1493] Fetching track data for Call Me...  done!
[16/1493] Fetching track data for Full Moon...  done!
[17/1493] Fetching tr

[133/1493] Fetching track data for Chasing Amy...  done!
[134/1493] Fetching track data for Forever Mine...  done!
[135/1493] Fetching track data for Comfortable...  done!
[136/1493] Fetching track data for Touch Me (feat. Kehlani) - Remix...  done!
[137/1493] Fetching track data for Skate Depot...  done!
[138/1493] Fetching track data for Cherry Ice Cream Sundae...  done!
[139/1493] Fetching track data for All My Girls Like To Fight...  done!
[140/1493] Fetching track data for HONEST...  done!
[141/1493] Fetching track data for Woman (feat. Lianne La Havas)...  done!
[142/1493] Fetching track data for Super Good...  done!
[143/1493] Fetching track data for Rare...  done!
[144/1493] Fetching track data for Treat...  done!
[145/1493] Fetching track data for Colors We Made...  done!
[146/1493] Fetching track data for don't waste your time...  done!
[147/1493] Fetching track data for Icarus...  done!
[148/1493] Fetching track data for SIA...  done!
[149/1493] Fetching track data for Red P

[268/1493] Fetching track data for Rewind...  done!
[269/1493] Fetching track data for Time Machine...  done!
[270/1493] Fetching track data for tsunami...  done!
[271/1493] Fetching track data for Should I? (Feat. Dopein)...  done!
[272/1493] Fetching track data for PLACE TO BE (feat. Francis)...  done!
[273/1493] Fetching track data for Good Morning...  done!
[274/1493] Fetching track data for sticker of your body...  done!
[275/1493] Fetching track data for Close to me...  done!
[276/1493] Fetching track data for COVID-19 (feat. SUPERBEE)...  done!
[277/1493] Fetching track data for UMMF...  done!
[278/1493] Fetching track data for you got me!...  done!
[279/1493] Fetching track data for Numbers (feat. CHANGMO)...  done!
[280/1493] Fetching track data for Room...  done!
[281/1493] Fetching track data for right now...  done!
[282/1493] Fetching track data for B.S. (feat. H.E.R.)...  done!
[283/1493] Fetching track data for All Me (feat. Keyshia Cole)...  done!
[284/1493] Fetching tra

[393/1493] Fetching track data for Dance Like No One’s Watching...  done!
[394/1493] Fetching track data for Always n Forever (feat. Lil Baby)...  done!
[395/1493] Fetching track data for So Done (feat. Khalid)...  done!
[396/1493] Fetching track data for Thought It'd Be Easy...  done!
[397/1493] Fetching track data for Do It (feat. Doja Cat, City Girls & Mulatto) [Remix]...  done!
[398/1493] Fetching track data for Air Forces...  done!
[399/1493] Fetching track data for Toxic...  done!
[400/1493] Fetching track data for Bop Bop...  done!
[401/1493] Fetching track data for Outta Lies...  done!
[402/1493] Fetching track data for No Scrubs...  done!
[403/1493] Fetching track data for Show Me...  done!
[404/1493] Fetching track data for Doo Wop (That Thing)...  done!
[405/1493] Fetching track data for Rock Your Body...  done!
[406/1493] Fetching track data for Pony...  done!
[407/1493] Fetching track data for No Diggity...  done!
[408/1493] Fetching track data for Ready or Not...  done!
[

[513/1493] Fetching track data for Run It! (feat. Juelz Santana)...  done!
[514/1493] Fetching track data for How Many Drinks?...  done!
[515/1493] Fetching track data for She's Playing Hard To Get...  done!
[516/1493] Fetching track data for Bartender...  done!
[517/1493] Fetching track data for Tell Me...  done!
[518/1493] Fetching track data for Don't Let Go (Love)...  done!
[519/1493] Fetching track data for Westside - Radio Edit...  done!
[520/1493] Fetching track data for How To Deal...  done!
[521/1493] Fetching track data for I'm Hot - Radio Edit...  done!
[522/1493] Fetching track data for Summertime - Single Edit...  done!
[523/1493] Fetching track data for Foolish...  done!
[524/1493] Fetching track data for Just Kickin' It...  done!
[525/1493] Fetching track data for I Wanna Know...  done!
[526/1493] Fetching track data for Superstar...  done!
[527/1493] Fetching track data for Want to Want Me...  done!
[528/1493] Fetching track data for Cry Me a River...  done!
[529/1493] 

...

[899/1493] Fetching track data for You and Me...  done!
[900/1493] Fetching track data for Don't Make Me...  done!
[901/1493] Fetching track data for Heartbeat...  done!
[902/1493] Fetching track data for All Eyes on Me...  done!
[903/1493] Fetching track data for Real One...  done!
[904/1493] Fetching track data for Should I...  done!
[905/1493] Fetching track data for Wildest Dreams...  done!
[906/1493] Fetching track data for Rotation...  done!
[907/1493] Fetching track data for DON't Believe IT...  done!
[908/1493] Fetching track data for Fallout City...  done!
[909/1493] Fetching track data for Thats What I Thought...  done!
[910/1493] Fetching track data for Body...  done!
[911/1493] Fetching track data for Never Be Yours...  done!
[912/1493] Fetching track data for Leave Me Alone...  done!
[913/1493] Fetching track data for HIT MY Line...  done!
[914/1493] Fetching track data for Stay...  done!
[915/1493] Fetching track data for Feelings or Faded...  done!
[916/1493] Fetching tr

[1040/1493] Fetching track data for Electric (feat. Khalid)...  done!
[1041/1493] Fetching track data for Alone...  done!
[1042/1493] Fetching track data for Call Out My Name...  done!
[1043/1493] Fetching track data for Collide (feat. Tyga)...  done!
[1044/1493] Fetching track data for Don't...  done!
[1045/1493] Fetching track data for Grass Ain't Greener...  done!
[1046/1493] Fetching track data for Hello Ego...  done!
[1047/1493] Fetching track data for Control Me...  done!
[1048/1493] Fetching track data for Beretta...  done!
[1049/1493] Fetching track data for Get You (feat. Kali Uchis)...  done!
[1050/1493] Fetching track data for Balcony...  done!
[1051/1493] Fetching track data for Twenty Ten...  done!
[1052/1493] Fetching track data for Nov 12...  done!
[1053/1493] Fetching track data for All You Need...  done!
[1054/1493] Fetching track data for Night Calls...  done!
[1055/1493] Fetching track data for Run My Mouth...  done!
[1056/1493] Fetching track data for Boundaries... 

[1170/1493] Fetching track data for Ghetto America...  done!
[1171/1493] Fetching track data for Can't Stop Lovin'...  done!
[1172/1493] Fetching track data for Room 148...  done!
[1173/1493] Fetching track data for Green Light 713...  done!
[1174/1493] Fetching track data for Differences...  done!
[1175/1493] Fetching track data for Panic...  done!
[1176/1493] Fetching track data for Outta My System...  done!
[1177/1493] Fetching track data for Suffocate - Superclean...  done!
[1178/1493] Fetching track data for Pullin' Me Back...  done!
[1179/1493] Fetching track data for Me & U...  done!
[1180/1493] Fetching track data for One Call Away...  done!
[1181/1493] Fetching track data for Walk Away (Remember Me) (feat. The DEY)...  done!
[1182/1493] Fetching track data for Rockin' That Sh**...  done!
[1183/1493] Fetching track data for Sexy Can I feat. Yung Berg...  done!
[1184/1493] Fetching track data for Be Without You - Kendu Mix...  done!
[1185/1493] Fetching track data for With You..

[1304/1493] Fetching track data for Deuces (feat. Tyga & Kevin McCall)...  done!
[1305/1493] Fetching track data for You...  done!
[1306/1493] Fetching track data for Falsetto...  done!
[1307/1493] Fetching track data for Dive In...  done!
[1308/1493] Fetching track data for Slow Down - 12" Version...  done!
[1309/1493] Fetching track data for Sunshine (feat. Lea)...  done!
[1310/1493] Fetching track data for In My Bed (DJ Tay James & Harv Unofficial Remix) [Dru Hill]...  done!
[1311/1493] Fetching track data for Feeling Good (AJDE Unofficial Remix) [Nina Simone]...  done!
[1312/1493] Fetching track data for Ain't No Mountain High Enough (Eclectic Method Unofficial Remix) [Marvin Gaye]...  done!
[1313/1493] Fetching track data for Ain't Nothing Like The Real Thing (DJ Rob Dinero Unofficial Remix) [Marvin Gaye & Tammi Terrell]...  done!
[1314/1493] Fetching track data for Do You Believe in Me...  done!
[1315/1493] Fetching track data for Hey...  done!
[1316/1493] Fetching track data for

[1436/1493] Fetching track data for TEXT (feat. Amindi)...  done!
[1437/1493] Fetching track data for Adorn...  done!
[1438/1493] Fetching track data for Don't...  done!
[1439/1493] Fetching track data for Best Part (feat. H.E.R.)...  done!
[1440/1493] Fetching track data for Boo'd Up...  done!
[1441/1493] Fetching track data for Thinkin Bout You...  done!
[1442/1493] Fetching track data for The Weekend...  done!
[1443/1493] Fetching track data for Cranes in the Sky...  done!
[1444/1493] Fetching track data for That's What I Like...  done!
[1445/1493] Fetching track data for Needed Me...  done!
[1446/1493] Fetching track data for Earned It (Fifty Shades Of Grey)...  done!
[1447/1493] Fetching track data for The Worst...  done!
[1448/1493] Fetching track data for Let It Burn...  done!
[1449/1493] Fetching track data for Turnin' Me Up...  done!
[1450/1493] Fetching track data for Deuces (feat. Tyga & Kevin McCall)...  done!
[1451/1493] Fetching track data for Really Love...  done!
[1452/

In [81]:
tracks_data_df = pd.concat(df_list)
tracks_data_df.head()

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,playlist_id,playlist_name
0,6nTcn8CybBcx0t7IaXEJ6O,Thank God I Found You (feat. Joe & Nas) - Make...,4iHNK0tOyZPYnBU7nGAgpQ,Mariah Carey,6zuCJaxmHKNKN5hMDF556U,251640,2000-04-18,55,0.608,0.760,...,-5.851,0,0.2420,0.0129,0.0,0.3320,0.366,168.211,6nTcn8CybBcx0t7IaXEJ6O,6nTcn8CybBcx0t7IaXEJ6O
0,7FWuzwapuKHTEyDJIcQOw3,Stutter (feat. Mystikal) - Double Take Remix,3zTOe1BtyTkwNvYZOxXktX,Joe,6zuCJaxmHKNKN5hMDF556U,213026,2000-04-18,55,0.767,0.759,...,-6.516,1,0.1170,0.0513,0.0,0.3100,0.677,89.989,7FWuzwapuKHTEyDJIcQOw3,7FWuzwapuKHTEyDJIcQOw3
0,3QE7ObGjkt1kYnhLvZPRUy,Go On Girl,21E3waRsmPlU7jZsS13rcj,Ne-Yo,52q9xNv8COtSU9phlbO5sg,261426,2007-01-01,65,0.637,0.521,...,-6.959,1,0.1090,0.1550,0.0,0.0945,0.562,83.896,3QE7ObGjkt1kYnhLvZPRUy,3QE7ObGjkt1kYnhLvZPRUy
0,6De63WRULDaa1fxOyLilnv,Stingy,7r8RF1tN2A4CiGEplkp1oP,Ginuwine,3t4OgAvvOzvXw8D2NHUeDR,259493,2003-04-08,54,0.622,0.573,...,-6.934,0,0.0547,0.2600,0.0,0.1730,0.617,127.132,6De63WRULDaa1fxOyLilnv,6De63WRULDaa1fxOyLilnv
0,08fhbfRSg7eEH0IZT7pB6a,That Girl,1goOx6gnQdUllLfSMsL4Rt,Marques Houston,4sAouJxcSwPIoT5dd6Qgj7,215960,2003-10-21,53,0.683,0.552,...,-5.139,1,0.1080,0.5480,0.0,0.2210,0.519,98.946,08fhbfRSg7eEH0IZT7pB6a,08fhbfRSg7eEH0IZT7pB6a


In [82]:
tracks_data_df.to_csv("data/"+KEYWORD+"_playlist_tracks_data.csv", index=False, encoding='utf-8')